In [1]:
import parse
import analyse
import matplotlib.pyplot as plt
import numpy as np

In [2]:
benchmarks = [
"results/ACASXU_run2a_1_1_batch_2000-retrain-1-0.05",
"results/ACASXU_run2a_1_1_batch_2000-student-1-0.05",
"results/ACASXU_run2a_1_2_batch_2000-retrain-1-0.05",
"results/ACASXU_run2a_1_2_batch_2000-student-1-0.05",
"results/mnist8x8_100_80_60_40_20_10_eps1-9000.2-15",
"results/mnist8x8_student_36_10-9000.7-top",
"results/mnist8x8_student_12_12_12_10-9000.7-top",
"results/mnist8x8_100_80_60_40_20_10-9200.3-top",
"results/mnist8x8_100_200_300_200_100_10-9200.2-top"
]
variants = [
    'REFINE_UNTIL_LAST_OPTIMISTIC1',
]
commit = '7417a86'
out_handlers=[parse.CegarOptimal]
err_handlers=[parse.RunLim]
both_handlers=[parse.NNEquivResultHandler]
num_runs=3

In [3]:
variant_runs = {}
for v in variants:
    variant_runs[v]={}
    for b in benchmarks:
        variant_runs[v][b]=[]
        for i in range(1,num_runs+1):
            try:
                variant_runs[v][b].append(
                    parse.BenchmarkRun(
                        "../"+b+"-"+v+"/"+str(i)+"/nnequiv-"+commit+"/stdout.log",
                        "../"+b+"-"+v+"/"+str(i)+"/nnequiv-"+commit+"/stderr.log",
                        out_handlers=out_handlers,
                        err_handlers=err_handlers,
                        both_handlers=both_handlers
                    )
                )
            except FileNotFoundError:
                print(f"Count not find "+b+"-"+v+"/"+str(i)+"/nnequiv-"+commit+"/stdout.log")

In [6]:
variants = [
    '',
    'noop'
]
commit = '04f3bfb'
out_handlers=[]
err_handlers=[parse.RunLim]
both_handlers=[parse.BueningResultHandler]
num_runs=3

In [7]:
for v in variants:
    variant_runs[v]={}
    for b in benchmarks:
        variant_runs[v][b]=[]
        for i in range(1,num_runs+1):
            try:
                variant_runs[v][b].append(
                    parse.BenchmarkRun(
                        "../"+b+"-"+v+"/"+str(i)+"/buening-"+commit+"/stdout.log",
                        "../"+b+"-"+v+"/"+str(i)+"/buening-"+commit+"/stderr.log",
                        out_handlers=out_handlers,
                        err_handlers=err_handlers,
                        both_handlers=both_handlers
                    )
                )
            except FileNotFoundError:
                print(f"Count not find "+b+"-"+v+"/"+str(i)+"/buening-"+commit+"/stdout.log")

In [8]:
variants = [
    'REFINE_UNTIL_LAST_OPTIMISTIC1',
    '',
    'noop'
]
variants_median = {}
for v in variants:
    variants_median[v]={}
    for b in benchmarks:
        variants_median[v][b]=parse.AggregateRun(variant_runs[v][b])

print(variants_median["REFINE_UNTIL_LAST_OPTIMISTIC1"]["results/ACASXU_run2a_1_1_batch_2000-retrain-1-0.05"].get_val(['runlim','real']))
print(variants_median[""]["results/ACASXU_run2a_1_1_batch_2000-retrain-1-0.05"].get_val(['runlim','real']))
print(variants_median["noop"]["results/ACASXU_run2a_1_1_batch_2000-retrain-1-0.05"].get_val(['runlim','real']))

1567.0
10800.1
10800.17


In [9]:
import pandas as pd

In [10]:
benchmarks_labels = {
"results/ACASXU_run2a_1_1_batch_2000-retrain-1-0.05":"ACAS_1_1-retrain",
"results/ACASXU_run2a_1_1_batch_2000-student-1-0.05":"ACAS_1_1-student",
"results/ACASXU_run2a_1_2_batch_2000-retrain-1-0.05":"ACAS_1_2-retrain",
"results/ACASXU_run2a_1_2_batch_2000-student-1-0.05":"ACAS_1_2-student",
"results/mnist8x8_100_80_60_40_20_10_eps1-9000.2-15":"MNIST_large-epsilon",
"results/mnist8x8_student_36_10-9000.7-top":"MNIST_small-top",
"results/mnist8x8_student_12_12_12_10-9000.7-top":"MNIST_medium-top",
"results/mnist8x8_100_80_60_40_20_10-9200.3-top":"MNIST_large-top",
"results/mnist8x8_100_200_300_200_100_10-9200.2-top":"MNIST_larger-top"
}

columns = ["Name", "Status", "Equiv", "NNEquiv", "Kleine Büning", "Kleine Büning (noop)"]
data = []
for b in benchmarks:
    cur_row = []
    cur_row.append(benchmarks_labels[b])
    cur_row.append("")
    cur_row.append("")
    for v in variants:
        cur_row[1]+="/"+str(variants_median[v][b].get_val(['result','status'],strategy='A')[0])
        cur_row[2]+="/"+str(variants_median[v][b].get_val(['result','is_equiv'],strategy='A')[0])
        cur_row.append(variants_median[v][b].get_val(['runlim','real']))
    data.append(cur_row)
print(columns)
table = pd.DataFrame(data, columns=columns)
table

['Name', 'Status', 'Equiv', 'NNEquiv', 'Kleine Büning', 'Kleine Büning (noop)']


,Name,Status,Equiv,NNEquiv,Kleine Büning,Kleine Büning (noop)
0,ACAS_1_1-retrain,/ok/out/out,/True/True/True,1567.00,10800.10,10800.17
1,ACAS_1_1-student,/ok/out/out,/True/True/True,350.30,10800.12,10800.13
2,ACAS_1_2-retrain,/ok/out/out,/True/True/True,2454.53,10800.07,10800.13
3,ACAS_1_2-student,/ok/ok/out,/True/False/True,347.22,322.12,10800.06
4,MNIST_large-epsilon,/ok/ok/ok,/True/False/False,142.98,19.62,114.09
5,MNIST_small-top,/ok/ok/ok,/True/True/True,37.80,3.51,2.79
6,MNIST_medium-top,/ok/ok/ok,/True/True/True,500.22,3.85,2.87
7,MNIST_large-top,/ok/ok/ok,/True/True/True,81.94,25.85,11.47
8,MNIST_larger-top,/out/ok/out,/True/True/True,10800.07,386.04,10800.06
